In [1]:
from selenium import webdriver;
from bs4 import BeautifulSoup;
from selenium.webdriver.chrome.service import Service;
from selenium.webdriver.common.by import By;
from selenium.webdriver.chrome.options import Options;
from selenium.webdriver.common.action_chains import ActionChains;
from webdriver_manager.chrome import ChromeDriverManager;
import time;
import pandas as pd;
import re;


In [2]:

# *open window

options  = Options();
options.add_experimental_option("detach", True);
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options);

driver.get('https://www.lamudi.com.ph/apartment/rent/'); # *link to open
driver.maximize_window(); # *maximize the window

# * data to get : 
# *title, location, price, key-info : bedroom, bath, floor area, agent


# ! step 1 : get max page and curr page


soup = BeautifulSoup(driver.page_source, "html.parser");
page = driver.find_element(By.CLASS_NAME, "nav-box-center").find_element(By.TAG_NAME ,'select');

curr_page = int(page.get_attribute('data-pagination-current'));
max_page = int(page.get_attribute('data-pagination-end'));

print(curr_page, max_page)
rows = [] 

# cnt = 0
while curr_page <= max_page:
    for item in soup.find_all('div', class_ = 'ListingCell-wrapper'): # selecting the card wrapper
        #getting listing details : title,loc,desc,price,feaures/ammenities,agent,agent_details
        title = item.find('h2',"ListingCell-KeyInfo-title").text.strip(); 
        location = item.find('span',"ListingCell-KeyInfo-address-text").text.strip();
        desc = item.find('div',"ListingCell-shortDescription").text.strip();
        price = item.select_one(".ListingCell-KeyInfo-price > :first-child").text.strip();
        features_holder = item.select(".ListingCell-keyInfo-details > .KeyInformation_v2 > .KeyInformation-attribute_v2");
        z = [];
        for x in features_holder: # looping through feautures
            # print(x)
            ammenity =  x.find('span', 'KeyInformation-label_v2').text.strip();
            amount = x.select_one("div > span").text.strip();
            y = ammenity + " : " + amount;
            print(y)
            z.append(y);
        features = z 

        agent = item.find('div',"ListingDetail-agent-name").text.strip();
        membership = item.find('div',"ListingDetail-agent-membership").text.strip();
        verified = item.find('img',"ListingDetail-agent-verified-icon");
        
        if(str(verified) != 'None'): # setting agent verified status
            verified = 'verified'
        else:
            verified = 'not verified'
        # print (verified)

        rows.append([title,location,desc,price,features,agent,membership,verified]) # appending to rows

    time.sleep(2);
    actions = ActionChains(driver);

    toAction = driver.find_element(By.CLASS_NAME,"LuxWidget-Title");
    actions.move_to_element(toAction).perform();

    time.sleep(2);
    actions = ActionChains(driver);
    try:
        nextbtn = driver.find_element(By.CSS_SELECTOR,".next > a");
        actions.click(nextbtn); 
        actions.perform();
    except:
        print('no button found')

    curr_page = curr_page + 1;
    time.sleep(5);


   

1 52
Bedroom : 1
Bath : 1
Floor area : 15 m²
Bedrooms : 40
Baths : 40
Floor area : 1700 m²
Bedrooms : 35
Baths : 35
Floor area : 1100 m²
Bedroom : 1
Bath : 1
Floor area : 26 m²
Bedrooms : 2
Bath : 1
Floor area : 36 m²
Bedrooms : 2
Bath : 1
Floor area : 80 m²
Bedroom : 1
Bath : 1
Floor area : 18 m²
Bedrooms : 8
Baths : 8
Floor area : 18 m²
Bedroom : 1
Bath : 1
Floor area : 15 m²
Bedroom : 1
Bath : 1
Floor area : 22 m²
Bedroom : 1
Bath : 1
Bedrooms : 2
Baths : 2
Floor area : 70 m²
Bedroom : 1
Bath : 1
Floor area : 15 m²
Bedrooms : 31
Bath : 1
Floor area : 950 m²
Bedroom : 1
Bath : 1
Bedroom : 1
Bath : 1
Floor area : 41 m²
Bedroom : 1
Bath : 1
Floor area : 22 m²
Bedroom : 1
Baths : 2
Floor area : 44 m²
Bedroom : 1
Bath : 1
Floor area : 45 m²
Bedroom : 1
Bath : 1
Floor area : 30 m²
Bedrooms : 12
Baths : 16
Floor area : 300 m²
Bedroom : 1
Bath : 1
Floor area : 47 m²
Bedroom : 1
Bath : 1
Bedroom : 1
Bath : 1
Floor area : 24 m²
Bedroom : 1
Baths : 2
Bedroom : 1
Bath : 1
Floor area : 23 m²
Bed

In [3]:
# rows
df = pd.DataFrame(rows,
                  columns=['title','location','description','price','features','agent','agent_membership','agent_verified'])

df

,title,location,description,price,features,agent,agent_membership,agent_verified
0,"Dormitory Spaces for Rent in Santo Tomas, Bata...","Santa Anastacia, Santo Tomas",Consuelo Park Residences (CPR) is a dormitory ...,Contact agent for price,"[Bedroom : 1, Bath : 1, Floor area : 15 m²]",Oasis FPIP,MEMBER SINCE 2019,verified
1,"Apartment Building For Rent Located in Molino,...","Molino VII, Bacoor",3 Story Apartment / Condo Building Along Molin...,Contact agent for price,"[Bedrooms : 40, Baths : 40, Floor area : 1700 m²]",Clement Dizon,MEMBER SINCE 2016,verified
2,Rush For Lease: 35-Door Staff House Near Circu...,"Olympia, Makati",Rush For Lease: 35-Door Staff House Near Circu...,"₱ 950,000","[Bedrooms : 35, Baths : 35, Floor area : 1100 m²]",Fern Marie Badong,MEMBER SINCE 2017,verified
3,Discounted Price! For Rent Studio Type Apartme...,"Maypajo, Caloocan","Looking for a new, clean, and affordable apart...","₱ 12,000","[Bedroom : 1, Bath : 1, Floor area : 26 m²]",JESSA MARTIN,MEMBER SINCE 2022,not verified
4,RDC Apartment for Rent located at 1156 (c) que...,"Ugong, Valenzuela",2 STOREY\n2 BEDROOM\n1 TOILET AND BATH\nRESIDE...,"₱ 7,000","[Bedrooms : 2, Bath : 1, Floor area : 36 m²]",ROSALIA CRAUWELS,MEMBER SINCE 2021,verified
...,...,...,...,...,...,...,...,...
1555,For Rent: Furnished Studio Unit at The Celerio...,"Tejeros, Makati","The Celerio is located at 4416 Montojo St., Ba...","₱ 14,000","[Bedroom : 1, Bath : 1, Floor area : 23 m²]",Len Magpusao,MEMBER SINCE 2021,verified
1556,For Rent Eisen Apartel Mla 200mbps nr SM San L...,"Santa Cruz, Manila","Rent : daily, weekly, monthly \nDaily: 1139\...","₱ 1,139","[Bedroom : 1, Bath : 1, Floor area : 12 m²]",Eisenbuilders Eisen Apartelle,MEMBER SINCE 2014,verified
1557,1-BR Apartment for Rent at Ferros Bel Air Towe...,"Poblacion, Makati",1-BR Apartment for Rent at Ferros Bel Air Towe...,"₱ 40,000","[Bedroom : 1, Bath : 1, Floor area : 60 m²]",Realus Realtors,MEMBER SINCE 2022,verified
1558,"Safestay Apartment for rent at Mactan Airport,...","Mactan, Lapu-Lapu",Fully Furnished apartment by GMRentals\n\nFloo...,"₱ 18,000","[Bedroom : 1, Bath : 1, Floor area : 36 m²]",Sheryl Dungog,MEMBER SINCE 2015,verified


In [4]:
import datetime

today_d =  datetime.datetime.now().strftime('%m-%d-%y')
file_name = 'lamudi-' + today_d + '.csv';
file_name


'lamudi-03-05-23.csv'

In [5]:
from pathlib import Path  
filepath = Path(r'C:\Users\Jake\Desktop\YEJI\\3Y2S\Analytics Techniques and Tools\WEB_CRAWLING\\appartment_rental\\'+file_name)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)
